# Rustpix Python Bindings Tutorial

This notebook demonstrates the Rustpix Python API for TPX3 processing.
It focuses on the SoA (structure-of-arrays) APIs and streaming workflows.


## Setup

This notebook assumes the `rustpix` Python package is installed.
If you are working from source, build the bindings with:

```bash
pixi run build
# or
maturin develop --release
```


In [ ]:
import rustpix

rustpix


## Read hits into SoA numpy arrays

`read_tpx3_file_numpy` returns a dictionary of numpy arrays, keeping data in SoA layout.


In [ ]:
path = "/path/to/data.tpx3"
hits = rustpix.read_tpx3_file_numpy(path)
hits.keys()


Expected fields:
- `x`, `y`, `tof`, `tot`, `timestamp`, `chip_id`
- `cluster_id` (only present if clustering has been run)


## Cluster hits (SoA path)

Use `cluster_hits_numpy` to label hits without constructing per-hit Python objects.


In [ ]:
config = rustpix.ClusteringConfig(radius=1.5, temporal_window_ns=1000, min_cluster_size=2)
labels, num_clusters = rustpix.cluster_hits_numpy(
    hits["x"],
    hits["y"],
    hits["tof"],
    hits["tot"],
    timestamp=hits["timestamp"],
    chip_id=hits["chip_id"],
    config=config,
    algorithm="grid",
)
num_clusters


## Extract neutrons (SoA path)

`extract_neutrons_numpy` returns a dictionary of numpy arrays describing neutrons.


In [ ]:
neutrons = rustpix.extract_neutrons_numpy(
    hits["x"],
    hits["y"],
    hits["tof"],
    hits["tot"],
    labels,
    num_clusters,
    timestamp=hits["timestamp"],
    chip_id=hits["chip_id"],
)
neutrons.keys()


## One-shot pipeline

`process_tpx3_file_numpy` performs read -> cluster -> extract and returns neutron arrays.


In [ ]:
neutrons_one_shot = rustpix.process_tpx3_file_numpy(
    path,
    config=config,
    algorithm="grid",
)
neutrons_one_shot.keys()


## Streaming processing

Use `MeasurementStream` to process large files incrementally. Each iteration yields
a dict with `hits` and `neutrons`.


In [ ]:
stream = rustpix.MeasurementStream(path, chunk_size=100_000_000, algorithm="grid")
for i, chunk in zip(range(3), stream):
    hits_chunk = chunk["hits"]
    neutrons_chunk = chunk["neutrons"]
    print(i, len(hits_chunk["x"]), len(neutrons_chunk["x"]))


## Optional: Arrow outputs

If `pyarrow` is installed, you can request Arrow tables directly.


In [ ]:
# hits_arrow = rustpix.read_tpx3_file_arrow(path)
# neutrons_arrow = rustpix.process_tpx3_file_arrow(path, config=config, algorithm="grid")
